In [30]:
from os import listdir
from sklearn.model_selection import train_test_split
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
%matplotlib inline
from keras.applications import VGG16,VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model

In [5]:
!echo "Number of Normal images"
!ls -l data/DATA_I_MADE/Images/Normal | egrep -c '^-'

Number of Normal images
541


In [6]:
!echo "Number of Trolled images"
!ls -l data/DATA_I_MADE/Images/Trolled | egrep -c '^-'

Number of Trolled images
557


In [7]:
Normal_Images = ['data/DATA_I_MADE/Images/Normal/'+i for i in listdir('data/DATA_I_MADE/Images/Normal')]
Trolled_Images = ['data/DATA_I_MADE/Images/Trolled/'+i for i in listdir('data/DATA_I_MADE/Images/Trolled')]
Y_normal = [0]*len(Normal_Images)
Y_trolled = [1]*len(Trolled_Images)
len(Normal_Images),len(Y_normal),len(Trolled_Images),len(Y_trolled)

(541, 541, 557, 557)

In [8]:
Images = Normal_Images+Trolled_Images
labels = Y_normal+Y_trolled
len(Images),len(labels)

(1098, 1098)

In [9]:
trainX, testX, trainY, testY = train_test_split(Images, labels, test_size=0.1, random_state=42, stratify=labels)
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.11, random_state=42, stratify=trainY)

In [19]:
def dump_to_pkl(file_location,item):
    with open(file_location, 'wb') as fp:
        pickle.dump(item, fp)

In [21]:
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/trainX.pickle',trainX)
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/testX.pickle',testX)
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/valX.pickle',valX)
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/trainY.pickle',trainY)
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/testY.pickle',testY)
dump_to_pkl('data/DATA_I_MADE/train_test_val_split/valY.pickle',valY)

In [10]:
def read_from_pkl(file_location):
    with open (file_location, 'rb') as fp:
        itemlist = pickle.load(fp)
    return itemlist

In [48]:
trainX = read_from_pkl('data/DATA_I_MADE/train_test_val_split/trainX.pickle')
testX = read_from_pkl('data/DATA_I_MADE/train_test_val_split/testX.pickle')
valX = read_from_pkl('data/DATA_I_MADE/train_test_val_split/valX.pickle')
trainY = read_from_pkl('data/DATA_I_MADE/train_test_val_split/trainY.pickle')
testY = read_from_pkl('data/DATA_I_MADE/train_test_val_split/testY.pickle')
valY = read_from_pkl('data/DATA_I_MADE/train_test_val_split/valY.pickle')

In [49]:
len(trainX),len(trainY),len(testX),len(testY),len(valX),len(valY)

(879, 879, 110, 110, 109, 109)

In [35]:
base_model = VGG19(weights='imagenet')

In [47]:
def read_and_preprocess(image_name):
    img = image.load_img(image_name, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [ ]:
trainX = [read_and_preprocess(i) for i in tqdm(trainX)]
testX = [read_and_preprocess(i) for i in tqdm(testX)]
valX = [read_and_preprocess(i) for i in tqdm(valX)]

In [42]:
model = Model(input=base_model.input, output=base_model.get_layer('fc2').output)
fc2_features = model.predict(img)

/home/hitkul/anaconda3/envs/nii_project/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """Entry point for launching an IPython kernel.


In [43]:
fc2_features.shape

(1, 4096)